In [2]:
import pandas as pd

In [58]:
data = pd.read_csv('鳥貴族.csv')

In [59]:
data

,ジャンル,メニュー名,カロリー,タンパク質,脂質,炭水化物,塩分,人権
0,貴族焼,もも貴族焼 たれ,350,26.3,21.6,9.5,1.9,3
1,貴族焼,もも貴族焼 塩,318,25.4,21.4,2.7,1.8,3
2,貴族焼,もも貴族焼 スパイス,328,25.7,21.5,4.6,4.2,3
3,貴族焼,むね貴族焼 たれ,253,32.6,8.9,8.5,1.4,3
4,貴族焼,むね貴族焼 塩,226,31.9,8.7,2.8,1.6,3
...,...,...,...,...,...,...,...,...
59,デザート,カタラーナアイス,247,2.5,13.5,29.9,0.1,-999
60,デザート,チョコパフェ -チュロ添え-,280,4.7,14.0,34.8,0.3,-999
61,デザート,魅惑のチョコレートみるく,332,7.9,15.4,43.0,0.3,-999
62,デザート,鳥貴ドームアイス ～いちご～,224,2.8,5.0,41.9,0.1,-999


In [39]:
import pulp

# 問題の定義
problem = pulp.LpProblem("Optimal_Menu_Selection", pulp.LpMinimize)

# 変数の定義
menu_vars = pulp.LpVariable.dicts("Menu", data.index, cat=pulp.LpBinary)

# 目的関数の定義
problem += pulp.lpSum([data.loc[i, 'カロリー'] * menu_vars[i] for i in data.index])

# 制約の追加
problem += pulp.lpSum([data.loc[i, 'タンパク質'] * menu_vars[i] for i in data.index]) >= 80
problem += pulp.lpSum([data.loc[i, '塩分'] * menu_vars[i] for i in data.index]) <= 10
problem += pulp.lpSum([data.loc[i, '炭水化物'] * menu_vars[i] for i in data.index]) <= 50
problem += pulp.lpSum([data.loc[i, '脂質'] * menu_vars[i] for i in data.index]) <= 30
problem += pulp.lpSum([data.loc[i, 'カロリー'] * menu_vars[i] for i in data.index]) <= 1000
# 新たな制約：人権の平均値が2を超える
problem += pulp.lpSum([data.loc[i, '人権'] * menu_vars[i] for i in data.index]) >= 30 * pulp.lpSum([menu_vars[i] for i in data.index])




# 最適化と複数解の列挙
solutions = []
nutritional_totals = []
max_solutions = 30 # 最大で見つけたい解の数
while len(solutions) < max_solutions:
    problem.solve()
    if pulp.LpStatus[problem.status] == 'Optimal':
        # 解の保存
        solution = [i for i in data.index if menu_vars[i].varValue == 1]
        solutions.append(solution)
        # 栄養価合計の計算
        total_calories = sum(data.loc[i, 'カロリー'] * menu_vars[i].varValue for i in data.index)
        total_protein = sum(data.loc[i, 'タンパク質'] * menu_vars[i].varValue for i in data.index)
        total_carbs = sum(data.loc[i, '炭水化物'] * menu_vars[i].varValue for i in data.index)
        total_fat = sum(data.loc[i, '脂質'] * menu_vars[i].varValue for i in data.index)
        total_salt = sum(data.loc[i, '塩分'] * menu_vars[i].varValue for i in data.index)
        total_human = sum(data.loc[i, '人権'] * menu_vars[i].varValue for i in data.index)
        nutritional_totals.append((total_calories, total_protein, total_carbs, total_fat,total_salt,total_human))
        # 解の除外制約を追加
        problem += pulp.lpSum([menu_vars[i] for i in solution]) <= len(solution) - 1
    else:
        break

# 結果の表示
for idx, sol in enumerate(solutions):
    print("Combination:", idx + 1)
    for i in sol:
        print(data.loc[i, 'メニュー名'])
    print("Nutritional Totals: Calories: {:.2f}, Protein: {:.2f}, Carbs: {:.2f}, Fat: {:.2f}, Salt: {:.2f}, 人間度: {:.2f}".format(
        nutritional_totals[idx][0], nutritional_totals[idx][1], nutritional_totals[idx][2], nutritional_totals[idx][3],nutritional_totals[idx][4],nutritional_totals[idx][5]))


Combination: 1
 砂ずり（砂肝）
 ひざなんこつ
 ささみ塩焼-わさび粗おろし添え-
 トリキの唐揚
Nutritional Totals: Calories: 544.00, Protein: 85.50, Carbs: 15.20, Fat: 13.10, Salt: 6.10, 人間度: 1006.00
Combination: 2
 砂ずり（砂肝）
 ささみ塩焼-わさび粗おろし添え-
 北海道産蛸わさび
 トリキの唐揚
Nutritional Totals: Calories: 556.00, Protein: 80.80, Carbs: 22.30, Fat: 13.10, Salt: 7.30, 人間度: 1004.00
Combination: 3
 砂ずり（砂肝）
 ささみ塩焼-わさび粗おろし添え-
 キャベツ盛
 トリキのチャンジャ
 トリキの唐揚
Nutritional Totals: Calories: 581.00, Protein: 82.00, Carbs: 28.20, Fat: 13.40, Salt: 9.00, 人間度: 1007.00
Combination: 4
 ささみ塩焼-わさび粗おろし添え-
 きも（レバー）
 トリキの唐揚
Nutritional Totals: Calories: 582.00, Protein: 85.30, Carbs: 19.40, Fat: 15.20, Salt: 5.40, 人間度: 1003.00
Combination: 5
 砂ずり（砂肝）
 ひざなんこつ
 きも（レバー）
 トリキの唐揚
Nutritional Totals: Calories: 584.00, Protein: 85.60, Carbs: 18.30, Fat: 16.10, Salt: 6.20, 人間度: 1004.00
Combination: 6
 むね貴族焼 スパイス
 ひざなんこつ
 トリキの唐揚
Nutritional Totals: Calories: 592.00, Protein: 80.00, Carbs: 18.50, Fat: 19.70, Salt: 8.50, 人間度: 1005.00
Combination: 7
 砂ずり（砂肝）
 きも（レバー）
 北海道産蛸わさび

In [51]:
import pulp

def solve_optimization_problem(data, constraints, objective, maximize=False, max_solutions=5):
    solutions = []
    nutritional_totals = []
    problem = pulp.LpProblem("Menu_Optimization", pulp.LpMaximize if maximize else pulp.LpMinimize)
    menu_vars = pulp.LpVariable.dicts("Menu", data.index, cat=pulp.LpBinary)

    # 目的関数
    if objective in data.columns:
        problem += pulp.lpSum([data.loc[i, objective] * menu_vars[i] for i in data.index])

    # 初期制約
    for nutrient, (constraint_type, value) in constraints.items():
        if nutrient in data.columns:
            if constraint_type == 'min':
                problem += pulp.lpSum([data.loc[i, nutrient] * menu_vars[i] for i in data.index]) >= value
            elif constraint_type == 'max':
                problem += pulp.lpSum([data.loc[i, nutrient] * menu_vars[i] for i in data.index]) <= value
            elif constraint_type == 'ave':
                problem += pulp.lpSum([data.loc[i, nutrient] * menu_vars[i] for i in data.index]) >= value * pulp.lpSum([menu_vars[i] for i in data.index])

    # 複数解の探索
    while len(solutions) < max_solutions:
        problem.solve()
        if pulp.LpStatus[problem.status] == 'Optimal':
            selected_items = [i for i in data.index if menu_vars[i].varValue == 1]
            solutions.append(selected_items)
            # 栄養価の計算
            total_calories = sum(data.loc[i, 'カロリー'] * menu_vars[i].varValue for i in selected_items)
            total_protein = sum(data.loc[i, 'タンパク質'] * menu_vars[i].varValue for i in selected_items)
            total_carbs = sum(data.loc[i, '炭水化物'] * menu_vars[i].varValue for i in selected_items)
            total_fat = sum(data.loc[i, '脂質'] * menu_vars[i].varValue for i in selected_items)
            total_salt = sum(data.loc[i, '塩分'] * menu_vars[i].varValue for i in selected_items)
            total_human = sum(data.loc[i, '人権'] * menu_vars[i].varValue for i in selected_items) / len(selected_items) if selected_items else 0
            nutritional_totals.append((total_calories, total_protein, total_carbs, total_fat, total_salt, total_human))
            # 解の除外制約を追加
            problem += pulp.lpSum([menu_vars[i] for i in selected_items]) <= len(selected_items) - 1
        else:
            break

    return solutions, nutritional_totals

# 結果の表示
constraints = {
    'タンパク質': ('min', 80),
    '塩分': ('max', 10),
    '炭水化物': ('max', 20),
    '脂質': ('max', 30),
    'カロリー': ('max', 1000),
    '人権': ('ave', 2)
}
solutions, nutritional_totals = solve_optimization_problem(data, constraints, 'カロリー', max_solutions=3)
for idx, sol in enumerate(solutions):
    print("Combination:", idx + 1)
    for i in sol:
        print(data.loc[i, 'メニュー名'])
    print("Nutritional Totals: Calories: {:.2f}, Protein: {:.2f}, Carbs: {:.2f}, Fat: {:.2f}, Salt: {:.2f}, Human Rights Average: {:.2f}".format(
        nutritional_totals[idx][0], nutritional_totals[idx][1], nutritional_totals[idx][2], nutritional_totals[idx][3], nutritional_totals[idx][4], nutritional_totals[idx][5]))


Combination: 1
 砂ずり（砂肝）
 ひざなんこつ
 ささみ塩焼-わさび粗おろし添え-
 トリキの唐揚
Nutritional Totals: Calories: 544.00, Protein: 85.50, Carbs: 15.20, Fat: 13.10, Salt: 6.10, Human Rights Average: 251.50
Combination: 2
 ささみ塩焼-わさび粗おろし添え-
 きも（レバー）
 トリキの唐揚
Nutritional Totals: Calories: 582.00, Protein: 85.30, Carbs: 19.40, Fat: 15.20, Salt: 5.40, Human Rights Average: 334.33
Combination: 3
 砂ずり（砂肝）
 ひざなんこつ
 きも（レバー）
 トリキの唐揚
Nutritional Totals: Calories: 584.00, Protein: 85.60, Carbs: 18.30, Fat: 16.10, Salt: 6.20, Human Rights Average: 251.00


In [81]:
import pulp

def solve_optimization_problem_with_penalty(data, constraints, objective, min_items=None, max_items=None,maximize=False, max_solutions=5):
    solutions = []
    nutritional_totals = []
    item_penalties = {i: 0 for i in data.index}  # 各アイテムのペナルティ初期化

    # 初期問題設定
    problem = pulp.LpProblem("Menu_Optimization", pulp.LpMaximize if maximize else pulp.LpMinimize)
    menu_vars = pulp.LpVariable.dicts("Menu", data.index, cat=pulp.LpBinary)

    # 繰り返し解を見つける
    while len(solutions) < max_solutions:
        # 目的関数（ペナルティを適用）
        if objective in data.columns:
            problem += pulp.lpSum([(data.loc[i, objective] - item_penalties[i]) * menu_vars[i] for i in data.index])

        # 制約の適用
        for nutrient, (constraint_type, value) in constraints.items():
            if nutrient in data.columns:
                if constraint_type == 'min':
                    problem += pulp.lpSum([data.loc[i, nutrient] * menu_vars[i] for i in data.index]) >= value
                elif constraint_type == 'max':
                    problem += pulp.lpSum([data.loc[i, nutrient] * menu_vars[i] for i in data.index]) <= value
                elif constraint_type == 'ave':
                    problem += pulp.lpSum([data.loc[i, nutrient] * menu_vars[i] for i in data.index]) >= value * pulp.lpSum([menu_vars[i] for i in data.index])
                elif constraint_type == 'cnt':
                    problem += pulp.lpSum([data.loc[i, nutrient] * menu_vars[i] for i in data.index]) <= value
            # アイテム数制約
        if min_items is not None:
            problem += pulp.lpSum(menu_vars[i] for i in data.index) >= min_items, "Min_Items"
        if max_items is not None:
            problem += pulp.lpSum(menu_vars[i] for i in data.index) <= max_items, "Max_Items"

        # 問題を解く
        problem.solve()
        if pulp.LpStatus[problem.status] == 'Optimal':
            selected_items = [i for i in data.index if menu_vars[i].varValue == 1]
            solutions.append(selected_items)
            # 栄養価の計算
            nutritional_totals.append({
                'Calories': sum(data.loc[i, 'カロリー'] * menu_vars[i].varValue for i in selected_items),
                'Protein': sum(data.loc[i, 'タンパク質'] * menu_vars[i].varValue for i in selected_items),
                'Carbs': sum(data.loc[i, '炭水化物'] * menu_vars[i].varValue for i in selected_items),
                'Fat': sum(data.loc[i, '脂質'] * menu_vars[i].varValue for i in selected_items),
                'Salt': sum(data.loc[i, '塩分'] * menu_vars[i].varValue for i in selected_items),
                'Human Rights': sum(data.loc[i, '人権'] * menu_vars[i].varValue for i in selected_items) / len(selected_items) if selected_items else 0
            })
            # 選択された商品に対してペナルティを増加
            for i in selected_items:
                item_penalties[i] -= data.loc[i, 'カロリー']   # ペナルティ値は調整可能
            # 次の問題インスタンスのために問題をリセット
            problem = pulp.LpProblem("Menu_Optimization", pulp.LpMaximize if maximize else pulp.LpMinimize)
        else:
            break

    return solutions, nutritional_totals

# 使用例と制約
constraints = {
    'タンパク質': ('min', 50),
    '塩分': ('max', 10),
    '炭水化物': ('max', 100),
    '脂質': ('max', 10),
    'カロリー': ('max', 1000),
    '人権': ('ave', 1.5)
}
solutions, nutritional_totals = solve_optimization_problem_with_penalty(data, constraints, 'カロリー', min_items=3, max_items=5, max_solutions=10)
for idx, solution in enumerate(solutions):
    print(f"Solution {idx + 1}:")
    for i in solution:
        print(f"  {data.loc[i, 'メニュー名']}")
    totals = nutritional_totals[idx]
    print(f"Nutritional Totals: Calories: {totals['Calories']:.2f}, Protein: {totals['Protein']:.2f}, Carbs: {totals['Carbs']:.2f}, Fat: {totals['Fat']:.2f}, Salt: {totals['Salt']:.2f}, Human Rights Average: {totals['Human Rights']:.2f}")


Solution 1:
   砂ずり（砂肝）
   ひざなんこつ
   ささみ塩焼-わさび粗おろし添え-
   北海道産蛸わさび
Nutritional Totals: Calories: 276.00, Protein: 50.60, Carbs: 9.10, Fat: 2.80, Salt: 5.20, Human Rights Average: 2.00
Solution 2:
   ささみ塩焼-わさび粗おろし添え-
   きも（レバー）
   キャベツ盛
   トリキのチャンジャ
Nutritional Totals: Calories: 339.00, Protein: 51.60, Carbs: 19.20, Fat: 5.20, Salt: 6.20, Human Rights Average: 2.00
Solution 3:
   むね貴族焼 塩
   ひざなんこつ
   キャベツ盛
   北海道産蛸わさび
   トリキのチャンジャ
Nutritional Totals: Calories: 389.00, Protein: 50.10, Carbs: 23.80, Fat: 9.90, Salt: 9.30, Human Rights Average: 2.20
Solution 4:
   砂ずり（砂肝）
   ささみ塩焼-わさび粗おろし添え-
   ホルモンねぎ盛ポン酢
Nutritional Totals: Calories: 322.00, Protein: 51.70, Carbs: 4.20, Fat: 9.10, Salt: 2.60, Human Rights Average: 2.00
Solution 5:
   ひざなんこつ
   ささみ塩焼-わさび粗おろし添え-
   むね明太マヨ焼
Nutritional Totals: Calories: 333.00, Protein: 54.20, Carbs: 3.90, Fat: 9.80, Salt: 3.10, Human Rights Average: 3.00
Solution 6:
   砂ずり（砂肝）
   ひざなんこつ
   きも（レバー）
   ひざなんこつ唐揚
Nutritional Totals: Calories: 386.00, Protein: 54.